In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:284: 

In [2]:
# SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)

In [3]:
feat_arr = [{'name':'hemoglobin', 'lower':12, 'upper':17.2, 'mean':10.3, 'std':2.3, 'min':(0, 5)},
            {'name':'ferritin', 'lower':10, 'upper':263, 'mean':697, 'std':3305, 'min':(1.6, 8)},
            {'name':'ret_count', 'lower':0.5, 'upper':2.5, 'mean':2.6, 'std':2.5, 'min':(0, 0.5)},
            {'name':'segmented_neutrophils', 'lower':0, 'upper':0, 'mean':0, 'std':2.4, 'min':(0, 0)},
            {'name':'tibc', 'lower':250, 'upper':450, 'mean':242, 'std':90, 'min':(3, 100)},
            {'name':'mcv', 'lower':80, 'upper':100, 'mean':90, 'std':8, 'min':(0, 65)}]

anem_feat_arr = [{'name':'hemoglobin', 'lower':12, 'upper':17.2, 'abnorm_lower':6, 'abnorm_upper':18},
                 {'name':'ferritin', 'lower':10, 'upper':263, 'abnorm_lower':0, 'abnorm_upper': 500},
                 {'name':'ret_count', 'lower':0.5, 'upper':2.5, 'abnorm_lower':0, 'abnorm_upper':6},
                 {'name':'segmented_neutrophils', 'lower':0, 'upper':0, 'abnorm_lower':0.1, 'abnorm_upper':7},
                 {'name':'tibc', 'lower':250, 'upper':450, 'abnorm_lower': 100, 'abnorm_upper':500},
                 {'name':'mcv', 'lower':80, 'upper':100, 'abnorm_lower': 75, 'abnorm_upper':105},
                 {'name':'gender', 'lower':0, 'upper':1}, #0=Female 1=male
                ]

#### The data

In [4]:
df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.183192,187.573466,4.951674,1.661383,316.593436,95.006424,181.242992,6.758433,60.586525,1,2.976104,282.291951,-1.000000,28.040619,96.721542,49.530542,23.186628,-1.000000,Hemolytic anemia
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,125.249617,6.350652,86.964793,1,2.968983,158.844879,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.000000,No anemia
2,12.749357,5.012158,3.502800,6.179371,498.418768,76.759285,159.834784,5.018156,67.065069,1,2.705719,442.293823,1.022939,56.850479,75.739552,72.072041,20.600875,44.872138,Iron deficiency anemia
3,11.508870,197.180945,1.200125,0.000000,457.033309,102.900301,131.177927,6.656823,66.403181,1,1.869338,223.896404,-1.000000,111.220307,66.999185,18.353272,14.132423,-1.000000,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Vitamin B12/Folate deficiency anemia


In [5]:
df.label.unique()

array(['Hemolytic anemia', 'No anemia', 'Iron deficiency anemia',
       'Unspecified anemia', 'Vitamin B12/Folate deficiency anemia',
       'Aplastic anemia', 'Anemia of chronic disease',
       'Inconclusive diagnosis'], dtype=object)

In [6]:
mult=4

#### 1 - Vitamin B12/Folate deficiecny anemia

In [7]:
vit_df = df[df.label=='Vitamin B12/Folate deficiency anemia']
vit_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,Vitamin B12/Folate deficiency anemia
6,6.939906,433.022945,-1.000000,3.831731,-1.000000,102.390026,-1.000000,3.588500,52.360187,1,2.896957,-1.000000,-1.000000,-1.000000,-1.000000,5.377878,-1.000000,-1.00000,Vitamin B12/Folate deficiency anemia
13,10.821902,134.578931,4.438313,3.022130,482.327091,101.919603,125.867191,4.693595,49.337676,0,0.810050,282.426797,-1.000000,100.431364,109.078360,63.046145,25.636901,-1.00000,Vitamin B12/Folate deficiency anemia
20,11.778437,191.937028,3.918479,1.881728,389.835900,101.602359,169.283019,5.828224,70.656023,1,0.864321,447.608725,0.807224,22.503145,98.530413,35.562075,12.959532,136.43449,Vitamin B12/Folate deficiency anemia
25,9.232404,-1.000000,-1.000000,0.709700,-1.000000,101.375414,109.743201,6.129433,46.584281,0,2.600773,321.032867,-1.000000,48.672338,67.691590,34.493253,26.806454,-1.00000,Vitamin B12/Folate deficiency anemia


In [8]:
for i in ['mcv', 'segmented_neutrophils']:
    vit_df[i] = vit_df[i].sample(frac=1-mult*0.05)
vit_df.isna().sum()

hemoglobin                  0
ferritin                    0
ret_count                   0
segmented_neutrophils    1800
tibc                        0
mcv                      1800
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [9]:
vit_df['mcv'] = vit_df['mcv'].fillna(random.uniform(95, 100))
vit_df['segmented_neutrophils'] = vit_df['segmented_neutrophils'].fillna(-1)
vit_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,Vitamin B12/Folate deficiency anemia
6,6.939906,433.022945,-1.000000,3.831731,-1.000000,102.390026,-1.000000,3.588500,52.360187,1,2.896957,-1.000000,-1.000000,-1.000000,-1.000000,5.377878,-1.000000,-1.00000,Vitamin B12/Folate deficiency anemia
13,10.821902,134.578931,4.438313,-1.000000,482.327091,98.197134,125.867191,4.693595,49.337676,0,0.810050,282.426797,-1.000000,100.431364,109.078360,63.046145,25.636901,-1.00000,Vitamin B12/Folate deficiency anemia
20,11.778437,191.937028,3.918479,1.881728,389.835900,101.602359,169.283019,5.828224,70.656023,1,0.864321,447.608725,0.807224,22.503145,98.530413,35.562075,12.959532,136.43449,Vitamin B12/Folate deficiency anemia
25,9.232404,-1.000000,-1.000000,0.709700,-1.000000,98.197134,109.743201,6.129433,46.584281,0,2.600773,321.032867,-1.000000,48.672338,67.691590,34.493253,26.806454,-1.00000,Vitamin B12/Folate deficiency anemia


In [10]:
vit_df.describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
count,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,9.486498,179.891842,0.819654,2.615692,108.191059,101.647174,84.761175,3.666338,45.005189,0.569444,0.814556,189.771251,-0.056218,47.301827,50.570266,28.600774,9.219000,23.574150
std,2.014827,165.540411,2.310386,2.537174,160.812083,2.150891,83.967674,2.541389,30.282889,0.495181,1.337818,160.248353,1.100644,50.362186,45.549528,26.814576,10.323252,43.131487
min,6.000016,-1.000000,-1.000000,-1.000000,-1.000000,98.197134,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,7.737610,-1.000000,-1.000000,0.524291,-1.000000,100.350804,-1.000000,2.842644,23.030772,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,9.458762,154.702565,-1.000000,2.650324,-1.000000,101.870074,68.447223,4.220283,46.574134,1.000000,0.928346,213.797783,-1.000000,31.751622,51.047896,24.564453,6.548631,-1.000000
75%,11.252786,325.713112,2.738242,4.810003,224.345084,103.429095,158.798852,5.622469,70.884520,1.000000,1.978088,329.420784,1.002219,90.816188,91.399064,51.879411,18.159897,47.067286
max,12.998632,499.921689,5.995977,6.998971,499.950779,104.999973,249.919719,6.999964,94.997996,1.000000,2.999253,449.976306,1.999873,149.997358,129.985350,79.975425,29.999063,139.777727


#### 2 - Unspecified anemia

In [11]:
unspec_df = df[df.label=='Unspecified anemia']
unspec_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
3,11.508870,197.180945,1.200125,0.0,457.033309,102.900301,131.177927,6.656823,66.403181,1,1.869338,223.896404,-1.000000,111.220307,66.999185,18.353272,14.132423,-1.000000,Unspecified anemia
9,11.134721,-1.000000,-1.000000,0.0,-1.000000,101.289788,130.274561,5.200105,89.077820,1,0.519895,246.999277,-1.000000,100.012213,51.498563,17.313546,10.708672,-1.000000,Unspecified anemia
14,11.267940,-1.000000,2.339196,0.0,-1.000000,102.086770,30.273153,3.301928,74.199225,0,0.769457,431.549854,0.409947,113.581821,109.533820,4.584599,14.719737,-1.000000,Unspecified anemia
19,9.683009,80.159398,0.128461,0.0,263.123104,100.680435,220.729300,6.598076,88.110782,1,2.703993,374.628666,0.271460,77.682348,129.169116,64.095456,15.580448,43.865898,Unspecified anemia
23,9.488839,484.020043,2.394415,0.0,201.992233,104.383813,231.105033,2.853901,23.541650,1,0.333596,412.656790,-1.000000,33.461756,125.852474,16.680947,21.532891,-1.000000,Unspecified anemia


In [12]:
for i in ['mcv', 'segmented_neutrophils']:
    unspec_df[i] = unspec_df[i].sample(frac=1-mult*0.05)
unspec_df.isna().sum()

hemoglobin                  0
ferritin                    0
ret_count                   0
segmented_neutrophils    1807
tibc                        0
mcv                      1807
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [13]:
unspec_df['mcv'] = unspec_df['mcv'].fillna(random.uniform(95, 100))
unspec_df['segmented_neutrophils'] = unspec_df['segmented_neutrophils'].fillna(-1)
unspec_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
3,11.508870,197.180945,1.200125,-1.0,457.033309,102.900301,131.177927,6.656823,66.403181,1,1.869338,223.896404,-1.000000,111.220307,66.999185,18.353272,14.132423,-1.000000,Unspecified anemia
9,11.134721,-1.000000,-1.000000,0.0,-1.000000,101.289788,130.274561,5.200105,89.077820,1,0.519895,246.999277,-1.000000,100.012213,51.498563,17.313546,10.708672,-1.000000,Unspecified anemia
14,11.267940,-1.000000,2.339196,0.0,-1.000000,102.086770,30.273153,3.301928,74.199225,0,0.769457,431.549854,0.409947,113.581821,109.533820,4.584599,14.719737,-1.000000,Unspecified anemia
19,9.683009,80.159398,0.128461,-1.0,263.123104,95.125054,220.729300,6.598076,88.110782,1,2.703993,374.628666,0.271460,77.682348,129.169116,64.095456,15.580448,43.865898,Unspecified anemia
23,9.488839,484.020043,2.394415,0.0,201.992233,104.383813,231.105033,2.853901,23.541650,1,0.333596,412.656790,-1.000000,33.461756,125.852474,16.680947,21.532891,-1.000000,Unspecified anemia


In [14]:
unspec_df.describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
count,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000
mean,9.486151,159.637895,1.873962,-0.200044,79.238365,101.030614,84.249717,3.640017,44.674830,0.559615,0.799378,186.134669,-0.067300,46.459278,49.546791,28.567087,9.235675,23.019935
std,2.005637,167.254559,2.326746,0.400055,144.648693,3.222187,84.877219,2.522200,30.252990,0.496461,1.339211,160.296620,1.097424,50.092761,45.406533,26.791500,10.384553,42.538870
min,6.001011,-1.000000,-1.000000,-1.000000,-1.000000,95.125054,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,7.748757,-1.000000,-1.000000,0.000000,-1.000000,100.339436,-1.000000,2.828110,22.834029,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,9.475567,109.999868,1.781250,0.000000,-1.000000,101.859731,65.040805,4.192058,45.972543,1.000000,0.916048,210.267636,-1.000000,30.514809,49.657314,24.797382,6.640985,-1.000000
75%,11.199575,305.127886,3.913500,0.000000,131.314287,103.445770,157.997541,5.559776,70.536261,1.000000,1.948408,326.227438,0.995074,89.874579,89.949700,51.789469,18.241435,46.127012
max,12.998757,499.561158,5.999136,0.000000,499.738036,104.999237,249.883733,6.999948,94.946984,1.000000,2.999584,449.978925,1.999928,149.998603,129.997970,79.990797,29.996488,139.970184


In [15]:
unspec_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
age                      0
gender                   0
indirect_bilirubin       0
transferrin              0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
label                    0
dtype: int64

#### 3 - Anemia of chronic disease

In [16]:
acd_df = df[df.label=='Anemia of chronic disease']
acd_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
10,11.135318,428.398182,-1.000000,-1.000000,209.146497,78.688528,164.515328,2.613933,52.107458,1,2.278890,247.051304,1.597835,89.407362,33.988481,35.890618,2.241372,58.013949,Anemia of chronic disease
17,10.199092,396.300771,-1.000000,0.383121,244.485759,76.122486,96.585180,4.259322,39.657844,1,1.755565,216.120079,-1.000000,132.068015,94.175429,42.028555,14.409510,-1.000000,Anemia of chronic disease
22,12.126436,211.275726,0.038403,3.319368,206.580036,75.160106,78.565027,4.975350,26.650942,1,1.509415,235.474768,0.657055,129.380934,111.086028,38.396648,19.505327,101.374596,Anemia of chronic disease
27,6.535154,240.978155,-1.000000,3.690522,148.114505,78.366106,193.077593,6.764819,18.866912,1,1.678183,352.632729,0.372056,127.141915,102.427321,24.867700,11.286109,109.990523,Anemia of chronic disease
33,9.080539,118.287498,-1.000000,6.426623,257.434925,79.861861,-1.000000,6.835960,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease


In [17]:
for i in ['mcv', 'ferritin', 'tibc']:
    acd_df[i] = acd_df[i].sample(frac=1-mult*0.033)
acd_df.isna().sum()

hemoglobin                  0
ferritin                 1288
ret_count                   0
segmented_neutrophils       0
tibc                     1288
mcv                      1288
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [18]:
acd_df['mcv'] = acd_df['mcv'].fillna(random.uniform(80, 85))
acd_df['ferritin'] = acd_df['ferritin'].fillna(random.uniform(27, 30))
acd_df['tibc'] = acd_df['tibc'].fillna(random.uniform(450, 460))
acd_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
10,11.135318,428.398182,-1.000000,-1.000000,209.146497,81.375147,164.515328,2.613933,52.107458,1,2.278890,247.051304,1.597835,89.407362,33.988481,35.890618,2.241372,58.013949,Anemia of chronic disease
17,10.199092,396.300771,-1.000000,0.383121,244.485759,76.122486,96.585180,4.259322,39.657844,1,1.755565,216.120079,-1.000000,132.068015,94.175429,42.028555,14.409510,-1.000000,Anemia of chronic disease
22,12.126436,211.275726,0.038403,3.319368,457.364712,75.160106,78.565027,4.975350,26.650942,1,1.509415,235.474768,0.657055,129.380934,111.086028,38.396648,19.505327,101.374596,Anemia of chronic disease
27,6.535154,240.978155,-1.000000,3.690522,148.114505,78.366106,193.077593,6.764819,18.866912,1,1.678183,352.632729,0.372056,127.141915,102.427321,24.867700,11.286109,109.990523,Anemia of chronic disease
33,9.080539,118.287498,-1.000000,6.426623,257.434925,79.861861,-1.000000,6.835960,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease


In [19]:
acd_df.describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
count,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000,9756.000000
mean,9.545389,212.688375,0.414354,2.286340,279.384383,78.013549,83.262390,3.665466,44.390412,0.568881,0.775897,187.393763,-0.068955,46.222987,50.358691,27.885954,9.125749,23.178494
std,2.016049,157.579332,2.158694,2.655859,132.890247,1.869887,83.695479,2.541390,30.547835,0.495258,1.353074,160.425471,1.101041,50.121552,45.694139,26.898381,10.323096,42.985072
min,6.000049,-1.000000,-1.000000,-1.000000,-1.000000,75.001008,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,7.840677,52.118046,-1.000000,-1.000000,180.019856,76.452295,-1.000000,2.846329,22.305801,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,9.536540,202.547388,-1.000000,2.196146,285.434424,77.858382,65.671875,4.233900,45.801206,1.000000,0.893610,212.828437,-1.000000,29.247477,51.303291,23.097128,6.342442,-1.000000
75%,11.300983,350.425219,1.777969,4.632642,396.800984,79.299927,156.791453,5.609224,70.309686,1.000000,1.960765,329.779614,0.985734,89.160722,91.055977,51.334097,18.257961,45.436594
max,12.998543,499.983220,5.995416,6.999746,457.364712,81.375147,249.886965,6.999592,94.980383,1.000000,2.999715,449.808088,1.999809,149.972185,129.998641,79.996134,29.995874,139.948106


In [20]:
acd_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
age                      0
gender                   0
indirect_bilirubin       0
transferrin              0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
label                    0
dtype: int64

#### 4 - Iron deficiency anemia

In [21]:
ida_df = df[df.label=='Iron deficiency anemia']
ida_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
2,12.749357,5.012158,3.502800,6.179371,498.418768,76.759285,159.834784,5.018156,67.065069,1,2.705719,442.293823,1.022939,56.850479,75.739552,72.072041,20.600875,44.872138,Iron deficiency anemia
7,7.299399,57.923466,4.354479,-1.000000,455.930031,78.721302,-1.000000,4.753823,87.508815,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Iron deficiency anemia
30,8.383559,5.983171,4.892520,3.444228,470.758184,77.466536,183.104614,4.159620,51.654358,1,0.004000,405.461045,-1.000000,55.449844,54.555321,4.529687,4.680840,-1.000000,Iron deficiency anemia
34,7.381669,64.669719,-1.000000,-1.000000,473.419233,78.938753,-1.000000,5.301884,88.433827,0,2.564396,-1.000000,-1.000000,-1.000000,-1.000000,55.942383,-1.000000,-1.000000,Iron deficiency anemia
48,12.577713,54.820500,5.418057,-1.000000,457.004738,75.536541,61.734053,3.767722,62.866585,1,0.316076,297.909421,1.145080,84.238050,71.089879,56.237142,12.416872,48.479169,Iron deficiency anemia


In [22]:
for i in ['mcv', 'ferritin', 'tibc']:
    ida_df[i] = ida_df[i].sample(frac=1-mult*0.033)
ida_df.isna().sum()

hemoglobin                  0
ferritin                 1223
ret_count                   0
segmented_neutrophils       0
tibc                     1223
mcv                      1223
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [23]:
ida_df['mcv'] = ida_df['mcv'].fillna(random.uniform(80, 85))
ida_df['ferritin'] = ida_df['ferritin'].fillna(random.uniform(100, 130))
ida_df['tibc'] = ida_df['tibc'].fillna(random.uniform(350, 450))
ida_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
2,12.749357,5.012158,3.502800,6.179371,498.418768,76.759285,159.834784,5.018156,67.065069,1,2.705719,442.293823,1.022939,56.850479,75.739552,72.072041,20.600875,44.872138,Iron deficiency anemia
7,7.299399,126.765387,4.354479,-1.000000,358.693883,78.721302,-1.000000,4.753823,87.508815,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Iron deficiency anemia
30,8.383559,5.983171,4.892520,3.444228,358.693883,77.466536,183.104614,4.159620,51.654358,1,0.004000,405.461045,-1.000000,55.449844,54.555321,4.529687,4.680840,-1.000000,Iron deficiency anemia
34,7.381669,64.669719,-1.000000,-1.000000,473.419233,78.938753,-1.000000,5.301884,88.433827,0,2.564396,-1.000000,-1.000000,-1.000000,-1.000000,55.942383,-1.000000,-1.000000,Iron deficiency anemia
48,12.577713,54.820500,5.418057,-1.000000,457.004738,75.536541,61.734053,3.767722,62.866585,1,0.316076,297.909421,1.145080,84.238050,71.089879,56.237142,12.416872,48.479169,Iron deficiency anemia


In [24]:
ida_df.describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
count,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000,9267.000000
mean,9.517013,54.569355,0.427623,1.041082,447.893515,78.269348,84.209498,3.668303,44.872910,0.562965,0.785082,187.031353,-0.048312,46.365508,49.920525,28.111622,9.120307,23.409107
std,2.009386,40.543639,2.175298,2.626415,83.071125,2.402834,84.446702,2.527143,30.600997,0.496046,1.340234,160.647801,1.100612,49.962108,45.412856,26.761747,10.311583,42.845338
min,6.001592,-1.000000,-1.000000,-1.000000,-1.000000,75.000253,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,7.783558,19.157554,-1.000000,-1.000000,455.668200,76.439559,-1.000000,2.826514,22.540751,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,9.530146,50.836917,-1.000000,-1.000000,470.567413,77.868307,65.445642,4.226119,46.447388,1.000000,0.904345,211.060062,-1.000000,30.484315,50.888598,23.774717,6.409995,-1.000000
75%,11.271834,83.685557,1.827993,3.133897,485.597462,79.313927,158.935664,5.598176,71.111691,1.000000,1.935088,330.972362,0.992870,89.418279,90.608491,51.623287,17.996956,47.055398
max,12.999167,126.765387,5.998681,6.998616,499.984787,83.383497,249.994895,6.999879,94.992216,1.000000,2.999959,449.795505,1.999921,149.995586,129.985036,79.972710,29.995382,139.926810


In [25]:
ida_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
age                      0
gender                   0
indirect_bilirubin       0
transferrin              0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
label                    0
dtype: int64

#### 5 - Hemolytic anemia

In [26]:
hem_df = df[df.label=='Hemolytic anemia']
hem_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.183192,187.573466,4.951674,1.661383,316.593436,95.006424,181.242992,6.758433,60.586525,1,2.976104,282.291951,-1.000000,28.040619,96.721542,49.530542,23.186628,-1.000000,Hemolytic anemia
5,8.536469,-1.000000,5.882326,-1.000000,257.240312,93.686821,-1.000000,6.652404,88.053129,1,-1.000000,-1.000000,1.847965,-1.000000,-1.000000,-1.000000,-1.000000,81.083195,Hemolytic anemia
15,6.192316,286.932759,2.457818,4.019337,188.956097,83.553418,180.073941,6.607154,58.749163,1,0.036139,171.140527,-1.000000,94.439122,79.577895,2.984295,29.515565,104.931325,Hemolytic anemia
21,9.238414,135.065898,2.473585,5.930673,232.824228,88.177683,24.716795,2.789811,79.551372,1,2.564055,312.212579,1.456958,102.176148,124.972771,24.278267,26.213768,54.466227,Hemolytic anemia
24,7.978067,-1.000000,4.402310,-1.000000,-1.000000,86.031312,195.739549,6.611103,59.586872,1,0.494394,304.607719,-1.000000,121.499726,128.081882,11.635735,1.018158,-1.000000,Hemolytic anemia


In [27]:
for i in ['mcv', 'ret_count']:
    hem_df[i] = hem_df[i].sample(frac=1-mult*0.05)
hem_df.isna().sum()

hemoglobin                  0
ferritin                    0
ret_count                1795
segmented_neutrophils       0
tibc                        0
mcv                      1795
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [28]:
hem_df['mcv'] = hem_df['mcv'].fillna(random.uniform(75, 80))
hem_df['ret_count'] = hem_df['ret_count'].fillna(random.uniform(1.5, 2))
hem_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.183192,187.573466,4.951674,1.661383,316.593436,77.109609,181.242992,6.758433,60.586525,1,2.976104,282.291951,-1.000000,28.040619,96.721542,49.530542,23.186628,-1.000000,Hemolytic anemia
5,8.536469,-1.000000,1.514899,-1.000000,257.240312,93.686821,-1.000000,6.652404,88.053129,1,-1.000000,-1.000000,1.847965,-1.000000,-1.000000,-1.000000,-1.000000,81.083195,Hemolytic anemia
15,6.192316,286.932759,2.457818,4.019337,188.956097,83.553418,180.073941,6.607154,58.749163,1,0.036139,171.140527,-1.000000,94.439122,79.577895,2.984295,29.515565,104.931325,Hemolytic anemia
21,9.238414,135.065898,2.473585,5.930673,232.824228,77.109609,24.716795,2.789811,79.551372,1,2.564055,312.212579,1.456958,102.176148,124.972771,24.278267,26.213768,54.466227,Hemolytic anemia
24,7.978067,-1.000000,4.402310,-1.000000,-1.000000,86.031312,195.739549,6.611103,59.586872,1,0.494394,304.607719,-1.000000,121.499726,128.081882,11.635735,1.018158,-1.000000,Hemolytic anemia


In [29]:
hem_df.describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
count,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000,8976.000000
mean,9.500730,110.430343,3.556463,0.638844,162.509600,87.441375,86.187883,3.628226,44.950753,0.576315,0.799106,190.789439,-0.050466,46.883783,50.446564,28.819956,9.366428,23.455645
std,2.024398,157.668881,1.440367,2.494780,173.564923,7.323027,84.598208,2.553736,30.200290,0.494169,1.336721,160.777837,1.098499,50.038040,45.228159,27.080726,10.400227,42.993476
min,6.000214,-1.000000,1.514899,-1.000000,-1.000000,77.109609,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,7.721399,-1.000000,2.335778,-1.000000,-1.000000,81.196486,-1.000000,2.809122,23.190715,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,9.489891,-1.000000,3.569312,-1.000000,129.648213,87.581969,70.876841,4.194366,46.668586,1.000000,0.923188,215.335152,-1.000000,30.613549,51.246214,24.942002,6.795334,-1.000000
75%,11.273937,218.951524,4.821224,2.266321,318.520326,93.859791,161.452599,5.597403,70.278145,1.000000,1.934164,333.201503,1.003649,90.121746,90.569093,52.199525,18.514494,47.277566
max,12.998570,499.897137,5.999972,6.999613,499.967563,99.993461,249.978161,6.999352,94.988381,1.000000,2.999028,449.941899,1.999669,149.968003,129.999938,79.998067,29.996980,139.995018


In [30]:
hem_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
age                      0
gender                   0
indirect_bilirubin       0
transferrin              0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
label                    0
dtype: int64

#### 6 - Aplastic anemia

In [31]:
aplastic_df = df[df.label=='Aplastic anemia']
aplastic_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
8,11.187990,164.100581,0.227583,3.968893,341.488393,96.078950,231.570282,3.243339,92.544523,1,2.044094,170.594778,1.682396,63.121036,111.657328,31.352594,26.531468,-1.000000,Aplastic anemia
16,11.356030,-1.000000,0.401737,6.897318,-1.000000,98.600152,36.092200,6.958929,50.643194,0,1.099612,210.594536,1.917605,94.215394,124.217847,30.423848,1.106421,126.263864,Aplastic anemia
18,12.575501,122.516705,0.815967,3.928216,114.116011,83.827078,151.561751,6.240497,61.386165,1,2.600698,178.169994,1.218809,26.154572,46.578234,79.080473,7.422325,-1.000000,Aplastic anemia
53,11.650012,-1.000000,1.301037,2.525998,-1.000000,91.482932,211.515134,5.779927,46.662258,0,2.023965,317.479506,-1.000000,135.117881,126.879905,29.766900,29.833623,-1.000000,Aplastic anemia
60,7.508734,-1.000000,0.645946,2.335338,277.369827,82.948806,249.842934,6.391393,86.477879,1,0.556199,428.452366,0.903946,132.301251,76.090767,40.424531,22.308847,67.135677,Aplastic anemia


In [32]:
for i in ['mcv', 'ret_count']:
    aplastic_df[i] = aplastic_df[i].sample(frac=1-mult*0.05)
aplastic_df.isna().sum()

hemoglobin                  0
ferritin                    0
ret_count                1804
segmented_neutrophils       0
tibc                        0
mcv                      1804
serum_iron                  0
rbc                         0
age                         0
gender                      0
indirect_bilirubin          0
transferrin                 0
creatinine                  0
cholestrol                  0
copper                      0
ethanol                     0
folate                      0
glucose                     0
label                       0
dtype: int64

In [33]:
aplastic_df['mcv'] = aplastic_df['mcv'].fillna(random.uniform(75, 80))
aplastic_df['ret_count'] = aplastic_df['ret_count'].fillna(random.uniform(2, 2.5))
aplastic_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
8,11.187990,164.100581,2.252678,3.968893,341.488393,96.078950,231.570282,3.243339,92.544523,1,2.044094,170.594778,1.682396,63.121036,111.657328,31.352594,26.531468,-1.000000,Aplastic anemia
16,11.356030,-1.000000,0.401737,6.897318,-1.000000,76.093190,36.092200,6.958929,50.643194,0,1.099612,210.594536,1.917605,94.215394,124.217847,30.423848,1.106421,126.263864,Aplastic anemia
18,12.575501,122.516705,0.815967,3.928216,114.116011,83.827078,151.561751,6.240497,61.386165,1,2.600698,178.169994,1.218809,26.154572,46.578234,79.080473,7.422325,-1.000000,Aplastic anemia
53,11.650012,-1.000000,1.301037,2.525998,-1.000000,76.093190,211.515134,5.779927,46.662258,0,2.023965,317.479506,-1.000000,135.117881,126.879905,29.766900,29.833623,-1.000000,Aplastic anemia
60,7.508734,-1.000000,0.645946,2.335338,277.369827,82.948806,249.842934,6.391393,86.477879,1,0.556199,428.452366,0.903946,132.301251,76.090767,40.424531,22.308847,67.135677,Aplastic anemia


#### Creating new dataframe

In [34]:
no_df = df[df.label=='No anemia']
inconc_df = df[df.label=='Inconclusive diagnosis']
len(no_df), len(inconc_df)

(10000, 4948)

In [35]:
combined_df = pd.concat([no_df, vit_df, unspec_df, acd_df, ida_df, hem_df, aplastic_df, inconc_df ], axis=0)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,12.190830,163.097819,3.084127,5.923027,232.339305,79.550470,-1.000000,3.748689,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease
1,9.944819,187.086684,5.096256,-1.000000,-1.000000,100.121185,-1.000000,5.857746,72.837294,0,1.146823,-1.000000,-1.000000,-1.000000,-1.000000,20.943783,-1.000000,-1.000000,Unspecified anemia
2,13.703345,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,80.372015,5.684361,45.702318,0,1.789854,206.817706,1.551467,90.217875,76.009442,22.651663,12.551513,-1.000000,No anemia
3,7.346123,27.669632,-1.000000,-1.000000,428.089083,77.375356,95.912445,6.469686,52.594562,0,0.222080,297.319109,1.265341,126.999491,46.998397,77.183582,8.518821,119.113878,Anemia of chronic disease
4,12.295548,225.097199,3.827717,0.000000,-1.000000,102.137453,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Unspecified anemia


In [36]:
utils.get_dt_performance(combined_df)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

(0.9495714285714286,
 0.9375796043606589,
 0.9649074036208941,
 datetime.timedelta(microseconds=2962))

In [37]:
combined_df.to_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.4.csv', index=False)